## IRS
(Import, Read, and Set Options)

In [1]:
import pandas as pd
from geopy.geocoders import GoogleV3
import numpy as np
import os

Data from: https://www.fairhousingelements.org/he-status

In [2]:
# df = pd.read_csv('Website View HE status.csv')
### Data from above link - but with coordinates
df = pd.read_csv('compliance_data_with_cords.csv')

## IQ
### Instructions and Questions

<strong>Requirements:</strong>

- Map showing which cities are compliant with their housing element, which ones are not, and which ones have NOT submitted a plan with a deadline approaching (three colors)

- Which cities had homes approved under the builder's element
     - Where can I find this? Is it in one of the reports?

- Pop-up/tooltip for cities to include data on housing element details
    - State mandated housing and affordable housing goals
    - What was accomplished during the last building cycle
        - Where can I find this? Is it in one of the reports?
- Pop-up/tooltip for builder's remedy (if a city falls out of compliance, builders can get any project approved if it has 20% affordability -- this has led to numerous projects)
    - Details about approved homes, and who the developer is
        - Where found?

## Analysis

In [25]:
df['HE Compliance'].value_counts()

IN                   165
OUT                  130
DUE                   13
ADOPTED-IN REVIEW      6
CONDITIONAL            1
Name: HE Compliance, dtype: int64

In [23]:
df['HE Status details'].value_counts()

Adopted & Reviewed    223
Draft Reviewed         41
Adopted                21
2nd Draft              14
NONE                    7
Draft                   2
Name: HE Status details, dtype: int64

## Make Map

In [44]:
### Make into two separate DFs ###
df_IN = df.loc[df['HE Compliance'] == 'IN']
df_OUT = df.loc[df['HE Compliance'] == 'OUT']
df_DUE = df.loc[(df['HE Compliance'] == 'ADOPTED-IN REVIEW') | (df['HE Compliance'] == 'DUE')]

In [12]:
df.columns

Index(['Unnamed: 0', 'Jurisdiction', 'County', 'HE Compliance', 'HE Due Date',
       'HE Status details', 'Adoption Date', 'COG', '6th Cycle Draft',
       'HCD Review Letter', 'Review Date', 'Population (2010)',
       'Median Income', 'geo_location', 'loc', 'point', 'lat', 'lon',
       'altitude', 'display_location'],
      dtype='object')

In [13]:
def popup_html(row):
    i = row
    location = df['display_location'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>{}'''.format(location) + '''</strong><br>
    </html>
    '''
    return html


In [19]:
data = {'color':['green','red','yellow'],
       'compliance_status':['IN','OUT','NO PLAN - Deadline Approaching']}
df_color_lg = pd.DataFrame(data)

In [49]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=4)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'CA Housing Element Map')

for i in range(0,len(df_IN)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=200, max_width=200)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup,icon=folium.Icon(color='green')).add_to(m)
    
for i in range(0,len(df_OUT)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=200, max_width=200)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup,icon=folium.Icon(color='red')).add_to(m)
    
for i in range(0,len(df_DUE)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=200, max_width=200)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup,icon=folium.Icon(color='orange')).add_to(m)
    
lgd_txt = '<span style="color: {col};">{txt}</span>'
    
for i, r in df_color_lg.iterrows():
    fg = folium.FeatureGroup(name=lgd_txt.format(txt=r['compliance_status'] + ' ' + '= ' + r['color'],col=r['color']))
    m.add_child(fg)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)

folium.TileLayer('Open Street Map',control=False).add_to(m)
    
folium.map.LayerControl('topleft', collapsed=False).add_to(m)

m

In [50]:
m.save('index.html')

## Map URL Snagger
Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [51]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [52]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CA_housing_element_map


## Get Coordinates for Map (done)

In [ ]:
# %store -r google_maps_API_Key
# geolocator = GoogleV3(api_key=google_maps_API_Key)
# df['geo_location'] = df['Jurisdiction'] + ' ' + df['County'] + ' CA'
# df['display_location'] = df['Jurisdiction'].str.capitalize() + ' in ' + df['County']
# df['loc'] = df['geo_location'].apply(geolocator.geocode, timeout=10)
# df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
# df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
# df.to_csv('compliance_data_with_cords.csv')